# Goal:

- provide a plaintext interface to analyze step-by-step what is happening in the encrypted code

- Used as a Python sanity check because I'm not that familiar with R.

In [7]:
import numpy as np
np.random.seed(42)
import pandas as pd

np.seterr(all='raise')

DEBUG = False

# Load and Process the Data

In [8]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def load_data(num_samples, compare_to_r_ref):
    x_file = "../data/X_norm_1024.csv"
    y_file = "../data/y_1024.csv"
    train_x = pd.read_csv(x_file)
    train_x = train_x.to_numpy()[:num_samples]
    train_y = pd.read_csv(y_file)
    train_y = train_y.to_numpy()[:num_samples]
    print(f"{bcolors.OKGREEN}Using subsampled data to compare Python-C++{bcolors.ENDC}")
    print(f"{bcolors.OKGREEN}Reading in {x_file}, {y_file} {bcolors.ENDC}")

    print(f"Train X shape is: {train_x.shape}")
    print(f"Train y shape is: {train_y.shape}")
    return train_x, train_y

In [9]:
NUM_SAMPLES = 1024
COMPARE_TO_R_REF = False
lr = 0.1
mu = 0.1
train_x, train_y = load_data(
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)

# Same shape as Marcelo's reference code
betas = np.zeros((10, ))

Using subsampled data to compare Python-C++
Reading in ../data/X_norm_1024.csv, ../data/y_1024.csv 
Train X shape is: (1024, 10)
Train y shape is: (1024, 1)


In [12]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas
    return np.expand_dims(sigmoid(preds), -1)

def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))
    t1 = np.matmul(-y.T, np.log(h))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]

def nesterov(betas, epochs, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    nesterov_loss = [0 for _ in range(epochs)]
    # for i in tqdm.trange(epochs):
    for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)
        phi_prime = theta - lr * np.squeeze(gradient)
        if i == 0:
            theta = phi_prime
        else:
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime
        loss = cost(train_x, train_y, theta)
        if DEBUG:
            print(f"Grad: {(gradient.squeeze() * lr).tolist()}")
            print(f"Theta: {theta.tolist()}")
            print(f"Phi: {phi.tolist()}")
        print(f"Iteration: {i} Loss: {loss}")
        nesterov_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return nesterov_loss, theta, phi


In [13]:
losses, theta, phi = nesterov(betas, 100, lr, mu, train_x, train_y)

Iteration: 0 Loss: 0.6824088332753198
Iteration: 1 Loss: 0.6713903103516063
Iteration: 2 Loss: 0.6610257231078792
Iteration: 3 Loss: 0.6513242754752047
Iteration: 4 Loss: 0.6422209559690284
Iteration: 5 Loss: 0.633655276525783
Iteration: 6 Loss: 0.6255762093407051
Iteration: 7 Loss: 0.6179406479255354
Iteration: 8 Loss: 0.6107117365232372
Iteration: 9 Loss: 0.6038575610703887
Iteration: 10 Loss: 0.5973501566062397
Iteration: 11 Loss: 0.5911647557546322
Iteration: 12 Loss: 0.5852792172730221
Iteration: 13 Loss: 0.5796735885054826
Iteration: 14 Loss: 0.5743297673054131
Iteration: 15 Loss: 0.5692312379069118
Iteration: 16 Loss: 0.564362861908849
Iteration: 17 Loss: 0.559710710508802
Iteration: 18 Loss: 0.555261927800757
Iteration: 19 Loss: 0.5510046176555508
Iteration: 20 Loss: 0.5469277486855726
Iteration: 21 Loss: 0.5430210732440103
Iteration: 22 Loss: 0.5392750574653908
Iteration: 23 Loss: 0.5356808201235694
Iteration: 24 Loss: 0.5322300786434356
Iteration: 25 Loss: 0.5289151010105486


In [6]:
losses[:10]

[0.6824088332753198, 0.6713903103516063, 0.6610257231078792]

In [6]:
0.4697637653026297
0.4697635335992973